# FM Demodulator on PYNQ

## Required Libraries

In this project, we use RTL2832 USB tuners to get our RF input samples. 
<img src="rtl2832.png" alt="RTL2832" style="width: 200px;"/>
In order to interact with this unit, you should install the following library:

https://pypi.org/project/pyrtlsdr/

In [6]:
from rtlsdr import *  # importing the library to interact with the RF tuner

sdr = RtlSdr()  # initializing an instance for the device

# configure device
sdr.sample_rate = 2.4e6  # reading 2.4M samples per second
sdr.center_freq = 94.1e6  # tuning to 94.1MHz
sdr.gain = 40.2

t_s = 1  # we read 1 second worth of samples
n_s = int(sdr.sample_rate)*t_s  # number of samples in 1 second

samples = sdr.read_samples(n_s)  # reading samples to the "samples" array
sdr.close()  # closing the device instance

In [7]:
import sk_dsp_comm.rtlsdr_helper as sdr
from IPython.display import Audio
import time

#samples = sdr.capture(1, fo=94.1e6, fs=2.4e6, gain=40.2)  # we already captured our samples,
#                                                            this is an alternative way for 
#                                                            gathering samples using sk_dsp_comm library
tic = time.clock()
z_bb, z_out = sdr.mono_FM(samples, fs=2.4e6, file_name='SW.wav')  # this line demodulates the samples and stores 
toc = time.clock()
print("Proceesing time: {} seconds".format(toc - tic))
#                                                             the output in SW.wav

Audio('SW.wav')  # play the results using your laptop speaker

Done!
Proceesing time: 5.144804000000022 seconds


In [8]:
from pynq import Overlay
from pynq import Xlnk

ol = Overlay('design_1.bit')
ol.download()

In [9]:
import asyncio
import pynq.lib.dma
import numpy as np

mfm_ip = ol.hier_0.FM_0
dma_ro = ol.hier_0.axi_dma_0
dma_i = ol.hier_0.axi_dma_1

length = 1000
length50 = 20
in_r = Xlnk().cma_array(shape=(length,), dtype=np.float32)
in_i = Xlnk().cma_array(shape=(length,), dtype=np.float32)
out_hw = Xlnk().cma_array(shape=(length50,), dtype=np.float32)
z_out_hw = np.zeros((2400*20,), dtype=np.float32)

async def write_ip():
    
    for i in range(2400):
        x = samples[i*1000:(i+1)*1000]
        x_hw = x.astype(np.complex64)

        np.copyto(in_r, np.real(x_hw))
        np.copyto(in_i, np.imag(x_hw))

        dma_ro.sendchannel.transfer(in_r)
        dma_i.sendchannel.transfer(in_i)

        mfm_ip.write(0x00, 1)

        dma_ro.sendchannel.wait()
        dma_i.sendchannel.wait()
        
        await asyncio.sleep(0.000001)

        in_r.flush()
        in_i.flush()

async def read_ip():
    
    for i in range(2400):
        dma_ro.recvchannel.transfer(out_hw)
        dma_ro.recvchannel.wait()
        await asyncio.sleep(0.000001)

        np.copyto(z_out_hw[i*20:(i+1)*20], out_hw)
        out_hw.flush()

loop = asyncio.get_event_loop()
write_task = asyncio.ensure_future(write_ip())
read_task = asyncio.ensure_future(read_ip())

tic = time.clock()
loop.run_until_complete(read_task)
toc = time.clock()
print("Proceesing time: {} seconds".format(toc - tic))

# Removing the IP task from the event loop.
write_task.cancel()
in_r.close()
in_i.close()
out_hw.close()

Proceesing time: 36.18695700000001 seconds


In [10]:
Audio(z_out_hw, rate=48000)